In [0]:
%%capture
!pip install pyro-ppl 

In [0]:
# General
import os
from collections import defaultdict
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline
# Pytorch
import torch
from torch.distributions import constraints
import torchvision.datasets as dset
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

# Pyro
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam, SGD

pyro.enable_validation(True)    # <---- This is always a good idea!
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)
# Enable smoke test - run the notebook cells on CI.
smoke_test = 'CI' in os.environ

# Setup dataloader

In [0]:
%%capture
!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
!unzip hymenoptera_data.zip

In [0]:
# for loading and batching MNIST dataset
def setup_data_loaders(batch_size=4, use_cuda=False):
    data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

    data_dir = 'hymenoptera_data'
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                              data_transforms[x])
                      for x in ['train', 'val']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                                shuffle=True, num_workers=4)
                  for x in ['train', 'val']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
    class_names = image_datasets['train'].classes
    return dataloaders['train'], dataloaders['val'], class_names

# Implement model

In [0]:
import torch.nn as nn
import torch.nn.functional as F

# Cifar
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        
#         return F.softmax(x,dim=1)

In [0]:
class Classifier(nn.Module):
    def __init__(self, use_cuda=False):
        super(Classifier, self).__init__()
        self.linear  = models.resnet18(pretrained=True)
        self.linear.fc = nn.Linear(self.linear.fc.in_features, 2)
        if use_cuda:
            self.cuda()
        self.use_cuda = use_cuda

    # define the model p(x|z)p(z)
    def model(self, x,y):
        # register PyTorch module `linear` with Pyro
        pyro.module("linear", self.linear)
        with pyro.plate("data", x.shape[0]):
            probs = self.linear(x)
            
            # score against actual images
            pyro.sample("obs", dist.Categorical(logits=probs).to_event(1), obs=y)

    # define the guide (i.e. variational distribution) q(z|x)
    def guide(self, x,y):   
        # pyro.module("linear", self.linear)
        pass

    def forward(self, x):
      return torch.argmax(self.linear(x),dim=1)

    

# Inference

## SVI

In [0]:
def train(svi, train_loader, use_cuda=False):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    acc = 0
    for x, y in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        # do ELBO gradient and accumulate loss
        epoch_loss += svi.step(x,y)
        acc += sum(y == classifier.forward(x))

    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train,acc*1.0/normalizer_train

In [0]:
def evaluate(svi, test_loader, use_cuda=False):
    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    acc=0
    for x, y in test_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
            y = y.cuda()
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x,y)
        acc += sum(y == classifier.forward(x))
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test,acc*1.0/normalizer_test

In [0]:
# Run options
LEARNING_RATE = 1.0e-3
USE_CUDA = True

# Run only for a single iteration for testing
NUM_EPOCHS = 1 if smoke_test else 30
TEST_FREQUENCY = 1

In [0]:
from pyro.infer import Trace_ELBO, JitTrace_ELBO, TraceEnum_ELBO, JitTraceEnum_ELBO, SVI

train_loader, test_loader,classes = setup_data_loaders(batch_size=64, use_cuda=USE_CUDA)

# clear param store
pyro.clear_param_store()

classifier = Classifier(use_cuda=USE_CUDA)

# setup the optimizer
adam_args = {"lr": LEARNING_RATE}
optimizer = Adam(adam_args)

# setup the inference algorithm
svi = SVI(classifier.model, classifier.guide, optimizer, loss=JitTrace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range(NUM_EPOCHS):
    total_epoch_loss_train,acc_train = train(svi, train_loader, use_cuda=USE_CUDA)
    train_elbo.append(-total_epoch_loss_train)
    print("[epoch %03d]  average training loss: %.4f ; acc : %.4f" % (epoch, total_epoch_loss_train,acc_train))

    if epoch % TEST_FREQUENCY == 0:
        # report test diagnostics
        total_epoch_loss_test,acc_test = evaluate(svi, test_loader, use_cuda=USE_CUDA)
        test_elbo.append(-total_epoch_loss_test)
        print("[epoch %03d] average test loss: %.4f ; acc : %.4f" % (epoch, total_epoch_loss_test,acc_test))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth



[epoch 000]  average training loss: 29.8026 ; acc : 0.9877
[epoch 000] average test loss: 29.2986 ; acc : 0.8431
[epoch 001]  average training loss: 14.3055 ; acc : 1.0000
[epoch 001] average test loss: 45.2841 ; acc : 0.8431
[epoch 002]  average training loss: 27.6476 ; acc : 0.9672
[epoch 002] average test loss: 33.8686 ; acc : 0.8431
[epoch 003]  average training loss: 24.5224 ; acc : 0.9426
[epoch 003] average test loss: 26.7088 ; acc : 0.8627
[epoch 004]  average training loss: 16.5513 ; acc : 0.9303
[epoch 004] average test loss: 54.5548 ; acc : 0.7843
[epoch 005]  average training loss: 16.3607 ; acc : 0.9344
[epoch 005] average test loss: 34.5489 ; acc : 0.8301
[epoch 006]  average training loss: 11.9025 ; acc : 0.9590
[epoch 006] average test loss: 27.4219 ; acc : 0.8431
[epoch 007]  average training loss: 10.2511 ; acc : 0.9713
[epoch 007] average test loss: 25.9186 ; acc : 0.8301
[epoch 008]  average training loss: 10.3689 ; acc : 0.9631
[epoch 008] average test loss: 23.04

In [0]:
acc = 0
for x, y in test_loader:
  x = x.cuda()
  y = y.cuda()
  acc += sum(y == classifier.forward(x))
  


In [0]:
print(acc *1.0 / 10000)

tensor(0.9748, device='cuda:0')


## MCMC (Draft)

In [0]:
x,y = train_loader.dataset

In [0]:
x.shape

torch.Size([1, 28, 28])

In [0]:
# for loading and batching MNIST dataset
def setup_data_loaders_mcmc( use_cuda=False):
    root = './data'
    download = True
    trans = transforms.ToTensor()
    train_set = dset.MNIST(root=root, train=True, transform=trans,
                           download=download)
    test_set = dset.MNIST(root=root, train=False, transform=trans)

    kwargs = {'num_workers': 1, 'pin_memory': use_cuda}
    train_loader = torch.utils.data.DataLoader(dataset=train_set,
        batch_size=60000, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(dataset=test_set,
        batch_size=10000, shuffle=False, **kwargs)
    return train_loader, test_loader

In [0]:
from pyro.infer.mcmc.api import MCMC
from pyro.infer.mcmc import NUTS
 
pyro.set_rng_seed(2)
classifier2 = Classifier()

# Mình chưa tìm được cách đẩy từ dataloader gốc về full data để pass mcmc 
# mà mình cũng không biết liệu có cách cho mcmc nhận input dạng batch không
# chắc là không
train_loader, test_loader = setup_data_loaders_mcmc()




In [0]:
x,y = next(iter(train_loader))



Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fea9c2d3780>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7feb46ec80f0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/l

In [0]:
x.shape

torch.Size([60000, 1, 28, 28])

In [0]:
kernel = NUTS(classifier2.model_mcmc,jit_compile=True, ignore_jit_warnings=True)
mcmc = MCMC(kernel, num_samples=3, warmup_steps=3)
mcmc.run(x.cpu(),y.cpu())


Warmup:   0%|          | 0/6 [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/pyro/primitives.py:371: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


In [0]:
posterior_samples = mcmc.get_samples()